$$ \huge{\text{ XGboost}}$$


 

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
np.random.seed(5)
import xgboost as xgb

In [4]:
#%pip install xgboost



<font size=4 >  [XGboost](https://www.kaggle.com/code/stuarthallows/using-xgboost-with-scikit-learn) is a very popular Classifier/Regressor.  However, it has a tendency to over fit.  Let's apply it to the MNIST
    data set.
    
https://xgboost.readthedocs.io/en/stable/tutorials/model.html

In [5]:
from sklearn.metrics import  confusion_matrix
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV, train_test_split

<font size=4 >  Load the MNIST data set from sklearn.

In [6]:
from sklearn.datasets import fetch_openml
X,y=fetch_openml('mnist_784', version=1, return_X_y=True, parser='pandas')

<font color =red size=4> Data Prep
   
 * do a train test split of the data     
 * perform a PCA on the dat set keeping 90% of the variance
   
 

In [7]:
X=X/255
X_train,X_t,y_train,y_t=train_test_split(X,y)
y_train,y_t=y_train.astype(int),y_t.astype(int)

from sklearn.decomposition  import PCA
pca=PCA()
pca=PCA(n_components=.9)# this tells PCA to preeserve 90% of the variance
X_train_reduced=pca.fit_transform(X_train)
X_t_reduced=pca.transform(X_t)


<font color =red size=4>  What is the original number of features and how many do you have left after PCA?

In [8]:
X.shape,X_train_reduced.shape

((70000, 784), (52500, 87))

<font color=brown>__Answer:__

<font size=4 color =red>Perform a GridSearchCV oh the training data using the following.
    
<font color=blue>     __NOTE: My training time on this was about 8 minutes!__</font>
    
    xgb_model = xgb.XGBClassifier(objective="multi:softprob", random_state=42)
    params = {
    "max_depth": [2, 6,10], # default 3
    "n_estimators": [100, 150], # default 100
    }

In [9]:
xgb_model = xgb.XGBClassifier(objective="multi:softprob", random_state=42)
params = {
    "max_depth": [2, 6,10], # default 3
    "n_estimators": [100, 150], # default 100
}
xgb_grid=GridSearchCV(xgb_model,params,cv=4,verbose = 0)

In [10]:
xgb_grid.fit(X_train_reduced,y_train)
print("best param",xgb_grid.best_params_,"bestscore",xgb_grid.best_score_)
DF=pd.DataFrame(xgb_grid.cv_results_)

best param {'max_depth': 6, 'n_estimators': 150} bestscore 0.9633904761904762


<font size=4 color =red>Display the results in a data frame. Include only the parameters, the mean_test_score and the rank_test_score.  

In [11]:

DF[["param_max_depth","param_n_estimators","mean_test_score","rank_test_score"]]

,param_max_depth,param_n_estimators,mean_test_score,rank_test_score
0,2,100,0.921181,6
1,2,150,0.932476,5
2,6,100,0.960933,2
3,6,150,0.963390,1
4,10,100,0.958476,4
5,10,150,0.960305,3


<font size=4 color =red>

<font size=4 color =red> Score the best_estimator on the __training set__ and produce a confusion matrix for the classifier using the __training data__. 

In [12]:
y_pred = xgb_grid.best_estimator_.predict(X_train_reduced)

print(confusion_matrix(y_train, y_pred))


[[5150    0    0    0    0    0    0    0    0    0]
 [   0 5910    0    0    0    0    0    0    0    0]
 [   0    0 5231    0    0    0    0    0    0    0]
 [   0    0    0 5402    0    0    0    0    0    0]
 [   0    0    0    0 5163    0    0    0    0    0]
 [   0    0    0    0    0 4724    0    0    0    0]
 [   0    0    0    0    0    0 5141    0    0    0]
 [   0    0    0    0    0    0    0 5419    0    0]
 [   0    0    0    0    0    0    0    0 5160    0]
 [   0    0    0    0    0    0    0    0    0 5200]]


In [13]:
xgb_grid.best_estimator_.score(X_train_reduced,y_train)

1.0

<font size=4 color =red>What do you expect is happening here?  What digit does it predict correctly the most?

<font color =brown size=4>__Answer:__

<font size=4 color =red> Score the best_estimator on the __test set__ and produce a confusion matrix for the classifier using the test data. 

In [14]:
xgb_grid.best_estimator_.score(X_t_reduced,y_t)

0.9688571428571429

In [15]:
y_t_pred = xgb_grid.best_estimator_.predict(X_t_reduced)

print(confusion_matrix(y_t, y_t_pred))

[[1729    0    4    1    2    2    6    2    6    1]
 [   1 1931    7    4    4    3    5    3    8    1]
 [   5    3 1702   15   10    0    3    8    7    6]
 [   3    1   16 1657    1   17    2   11   15   16]
 [   4    4    4    0 1606    2    3    3    4   31]
 [   3    1    4   14    5 1526   17    1   12    6]
 [  11    1    0    1    3   18 1696    0    5    0]
 [   2   10   11    2    5    0    0 1824    8   12]
 [   4    3    6   12    6   12    3    3 1610    6]
 [   4    6    2   12   25    9    1   10   15 1674]]


<font size=4 color =red> Does the confirm your assumptions from above? Why?

<font color =brown size=4>__Answer:__